In [2]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [3]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [4]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-distdeck.pth"
)

In [5]:
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck2.pth")

In [6]:
df = pd.read_csv("data/ori/test.csv")

In [7]:
distdeck["dist"].shape, distdeck["level"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [8]:
distdeck.keys(), deck2.keys()

(dict_keys(['dist', 'level']), dict_keys(['feat', 'otext', 'fclevel', 'ids']))

In [9]:
total_len = 1418916

In [110]:
def policy(dists, tlevels, fclevel):
    # tlevels는 다 같은데, tlevels랑 fclevel이 다른 경우도 7임.
    # 그 외에 telvels[0]이 0.5보다 크면 7임.
    # 다만 fclevel이 3, 5일 경우에는 tlevels[0]이 1.2보다 커야지 7임.

    is_same = (tlevels == tlevels[0]).all()
    if fclevel in [2, 4, 6]:
        return fclevel.item()
    if fclevel == 3 or 3 in tlevels:
        return fclevel.item() if dists[0] < 0.5 else 7
    if fclevel == 5 or (is_same and tlevels[0] == 5):
        return fclevel.item() if dists[0] < 1.5 else 7
    if fclevel == 1 and 1 in tlevels:
        return fclevel.item() if dists[0] < 0.8 else 7
    if fclevel == 0 and 0 in tlevels:
        return fclevel.item() if dists[0] < 0.3 else 7
    return 7

In [111]:
out_dists = [None for _ in range(total_len)]
out_levels = [None for _ in range(total_len)]
out_fclevels = [None for _ in range(total_len)]
N = distdeck["dist"].size(0)
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["level"][i]
        fclevel = deck2["fclevel"][i]
        out_level = policy(dists, levels, fclevel)
        ids = deck2["ids"][i]
        for j in ids:
            out_levels[j - 1000000] = out_level
            out_dists[j - 1000000] = dists
            out_fclevels[j - 1000000] = fclevel
        t.update()

100%|███████████████████████████████████████████████████| 1095951/1095951 [03:01<00:00, 6025.68it/s]


In [112]:
out_levels = np.array(out_levels)

In [113]:
# 각 레벨별 개수 0.5이하일 때 fclevel을 쓰기로
for i in range(8):
    cnt = (out_levels == i).sum()
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 1003660 70.73%
1 : 395017 27.84%
2 : 42 0.00%
3 : 12878 0.91%
4 : 34 0.00%
5 : 6334 0.45%
6 : 31 0.00%
7 : 920 0.06%


In [115]:
(out_levels == 7).nonzero()[0][:100]

array([  3780,   6003,   7890,   8699,   9928,  11151,  11515,  12250,
        15160,  16932,  18788,  19574,  19856,  22246,  23836,  26611,
        27854,  28972,  29461,  30852,  37174,  37539,  38180,  38592,
        41200,  44077,  45615,  46047,  50471,  50899,  51887,  51953,
        53736,  54247,  56120,  56488,  58126,  58797,  61121,  61129,
        64630,  65387,  66248,  66649,  68203,  68300,  69197,  69985,
        70089,  72520,  72550,  73672,  74637,  76838,  79586,  79909,
        81139,  82047,  82287,  82598,  84329,  85474,  88037,  89806,
        93830,  93934,  97456,  98325, 100362, 100875, 101715, 102363,
       103617, 107200, 110013, 112430, 113459, 113813, 117120, 117614,
       119211, 119436, 119759, 120333, 122720, 124131, 131484, 133353,
       133911, 134165, 137872, 138723, 141569, 143352, 144966, 146134,
       146998, 147281, 148482, 148957])

In [15]:
xdf = pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv")

In [16]:
xlevels = xdf.level.to_numpy()

In [64]:
(xlevels == 7).nonzero()[0][:100]

array([  6003,   7890,   9928,  11151,  11515,  15160,  16932,  19574,
        22246,  26611,  27854,  30852,  37174,  38180,  38592,  41200,
        50471,  50899,  51887,  51953,  53736,  54247,  56488,  58126,
        58797,  61121,  64630,  65387,  66248,  66649,  68203,  68300,
        69985,  70089,  74637,  76838,  79586,  79909,  81139,  82047,
        82287,  84329,  85474,  88037,  89806,  93830,  93934,  97456,
        98325, 100362, 100875, 101297, 102363, 103617, 113459, 117120,
       117614, 119211, 119436, 119759, 122720, 124131, 133353, 133911,
       134165, 137872, 138723, 143352, 144966, 146134, 146998, 147281,
       148482, 148979, 152604, 153382, 153891, 158447, 159643, 160810,
       165543, 169905, 170457, 175138, 182070, 182445, 184448, 190307,
       193314, 194715, 198598, 201568, 202879, 203311, 203669, 208260,
       209443, 210211, 210933, 213041])

In [118]:
df.full_log[12250]

'oscap: msg: "xccdf-result", scan-id: "0001605230036", content: "ssg-centos-7-ds.xml", title: "Verify and Correct File Permissions with RPM", id: "xccdf_org.ssgproject.content_rule_rpm_verify_permissions", result: "fail", severity: "high", description: "Discretionary access control is weakened if a user or group has access permissions to system files and directories greater than the default. The RPM package management system can check file access permissions of installed software packages, including many that are important to system security. Verify that the file permissions, ownership, and gruop membership of system files and commands match vendor values. Check the file permissions, ownership, and group membership with the following command: $ sudo rpm -Va | grep \'^.M\' Output indicates files that do not match vendor defaults. After locating a file with incorrect permissions, run the following command to determine which package owns it: $ rpm -qf FILENAME Next, run the following comm

In [22]:
tdf = pd.read_csv("data/ori/train.csv")

In [23]:
def find_all(text):
    cnt = 0
    for i in range(len(tdf)):
        if text in tdf.full_log[i]:
            print(f"{i:8} | {tdf.level[i]} | {tdf.full_log[i]}")
            cnt += 1
            if cnt > 5:
                break

In [91]:
find_all("xccdf-result")

    3396 | 3 | oscap: msg: "xccdf-result", scan-id: "0001603070557", content: "ssg-centos-7-ds.xml", title: "Record Events that Modify the System's Discretionary Access Controls - setxattr", id: "xccdf_org.ssgproject.content_rule_audit_rules_dac_modification_setxattr", result: "fail", severity: "low", description: "At a minimum, the audit system should collect file permission changes for all users and root. If the auditd daemon is configured to use the augenrules program to read audit rules during daemon startup (the default), add the following line to a file with suffix .rules in the directory /etc/audit/rules.d: -a always,exit -F arch=b32 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the system is 64 bit then also add the following line: -a always,exit -F arch=b64 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the auditd daemon is configured to use the auditctl utility to read audit rules during daemon startup, add the following line to /etc/audit

---

In [49]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [50]:
out_df = {"id": out_ids, "level": out_levels}

In [51]:
out_df = pd.DataFrame(out_df)

In [52]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [47]:
# 0.9198
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver1.csv", index=False
)

In [17]:
# ver1보다 threshold를 좀 더 만만하게 준 버전 - 0.9207
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv", index=False
)

In [22]:
# level 3, 5를 꼭 모두 3, 5이지 않아도 되게 설정 - 0.9189
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver3.csv", index=False
)

In [53]:
# level 3, 5를 꼭 모두 3, 5이지 않아도 되게 설정 - 0.9189
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver4.csv", index=False
)